# CERR Segmentation Routines in Slicer 

### Pull imaging data from Slicer viewer and import to CERR for DL segmentation analysis

Setup and module load for Octave oct2py interface

In [ ]:
import os
import numpy as np

In [ ]:
octave_cli_path = 'C:/Program Files/GNU Octave/Octave-6.2.0/mingw64/bin/octave-cli.exe'
os.environ['OCTAVE_EXECUTABLE'] = octave_cli_path

In [ ]:
from oct2py import octave
from oct2py import Struct

In [ ]:
#import octave add-on packages
octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')

In [ ]:
javaPath = 'C:/Program Files/Java/jre1.8.0_291/bin/server'
octave.setenv('JAVA_HOME',javaPath)

Pull imaging data from the Slicer MRML scene and import to CERR

In [ ]:
#Get preloaded DICOM volume node from MRML scene
volumeNode = slicer.util.getNode('*CT*')

In [ ]:
#Get 3x3 voxel matrix from volume node
imgVolume = np.copy(slicer.util.arrayFromVolume(volumeNode))

#Origin, image spacing
imgOrigin = volumeNode.GetOrigin()
imgSpacing = volumeNode.GetSpacing()

#Get affine matrix with direction cosines, pixdim, origin
imgMatrixVtk = vtk.vtkMatrix4x4()
volumeNode.GetIJKToRASMatrix(imgMatrixVtk)

imgMatrix = np.eye(4)
imgMatrixVtk.DeepCopy(imgMatrix.ravel(),imgMatrixVtk)

imgData = volumeNode.GetImageData()
dtype = imgData.GetScalarSize()

In [ ]:
#add CERR code to Octave path

localPathToCERR = 'Y:\Eve\sliCERR\CERR'
octave.addpath(octave.genpath(localPathToCERR));

In [ ]:
infoS = Struct()
infoS.Offset = np.array(imgOrigin) #is this different from the origin?
infoS.PixelDimensions = np.array(imgSpacing)
infoS.Dimensions = np.array(imgVolume.shape)
scanOffset = 0
scanName = 'slicer_import'

In [ ]:
imV2 = octave.permute(imgVolume,[2,3,1])

In [ ]:
planC = octave.mha2cerr(infoS, imV2,scanOffset,scanName,[],0)

In [ ]:
octave.push('planC',planC)

In [ ]:
octave.eval('indexS = planC{end}')

Run Segmentation

In [ ]:
planC = runSegForPlanCInCondaEnv(planC,sessionPath,algorithm,condaEnvName);

In [ ]:
octave.eval('init_ML_DICOM')

In [ ]:
segdir = 'Y:\Eve\sliCERR\seg_sessions'
octave.push('segdir',segdir)

condaEnvName = 'Y:\Eve\sliCERR\CT_heart_sub_structs'
octave.push('condaEnvName',condaEnvName)

algorithm = 'CT_Pericardium_DeepLab'
octave.push('algorithm',algorithm)

In [ ]:
octave.exec('planC = runSegForPlanCInCondaEnv(planC,sessionPath,algorithm,condaEnvName)')

In [65]:
octave.save_planC(planC,[],'passed','slicer_0617.mat')

CERR>>  Saved slicer_0617.mat. Ready. (13-Jul-2021 12:16:48)


Cell([[{'timeSaved': Cell(['13-Jul-2021 11:43:17']), 'lastSavedInVer': '6.0'},
        0x0 StructArray containing the fields:
    imageNumber
    imageType
    caseNumber
    patientName
    writer
    commentDescription
    dateWritten
    fileOfOrigin
    unitNumber
    headInOut
    comment,
        {'scanArray': array([[[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        ...,
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.]],

       [[-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        ...,
        [-1024., -1024., -1024., ..., -1024., -1024., -1024.],
        [-1024., -1024.,

Visualize PERICARDIUM structure in Slicer

In [ ]:
octave.eval('structureListC = {planC{indexS.structures}.structureName};')
strName = 'PERICARDIUM'
octave.push('strName',strName)
octave.eval("structNum = getMatchingIndex(lower(strName),lower(structureListC),'exact');")
octave.eval('mask3M = getStrMask(structNum, planC);')
mask3M = octave.pull('mask3M')

In [ ]:
##set up segmentation nodes
segLabelString = 'PERICARDIUM'
segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segLabelString)
segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(volumeNode)